# Mapping quality check

<big><b>File under construction... </b></big> (05/21/2021) <br>

---

## <b>Preparing session for IFB core cluster</b>

<em>loaded JupyterLab</em> : Version 2.2.9

In [1]:
echo "=== Cell launched on $(date) ==="

echo "=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ==="
jobid=$(squeue -hu $USER | awk '/jupyter/ {print $1}')
sacct --format=JobID,AllocCPUS,NODELIST -j ${jobid}

echo "=== Working's root folder is ==="
gohome="/shared/projects/gonseq/Building/" # to adjust with your project's folder
echo "${gohome}"
echo ""

echo "=== current folder tree ==="
tree d -L 2 "${gohome}"
echo "=== current working directory ==="
echo "${PWD}"

=== Cell launched on Thu May 27 10:53:34 CEST 2021 ===
=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ===
       JobID  AllocCPUS        NodeList 
------------ ---------- --------------- 
16935772             10     cpu-node-41 
16935772.ba+         10     cpu-node-41 
16935772.0           10     cpu-node-41 
=== Working's root folder is ===
/shared/projects/gonseq/Building/

=== current folder tree ===
d [error opening dir]
/shared/projects/gonseq/Building/
├── Data
│   ├── fastq
│   ├── info
│   ├── sra
│   ├── sra-files_creation_fastqgz.log
│   ├── sra-files_integrity.log
│   └── sra-files_retrieval.log
├── Pipeline
│   ├── Pipe_1-bash_download-files.ipynb
│   ├── Pipe_2-bash_raw-data-quality.ipynb
│   ├── Pipe_3-bash_preprocessing-and-verify.ipynb
│   ├── Pipe_4-bash_classical-reads-mapping.ipynb
│   └── Pipe_5-bash_mapping-quality.ipynb
├── Reference
│   ├── extracted
│   ├── gencode.v37.primary_assembly.annotation.gtf.gz
│   ├── GRCh38.primary_assembly.g

In [3]:
module load samtools qualimap multiqc

echo "===== quality control & bam sorting ====="  # I changed it after I used it
samtools --version | head -n 2
echo "===== graphical quality controls ====="  # I changed it after I used it
qualimap --help | head -n 4
echo "===== quality reports compilation ====="
multiqc --version

===== quality control =====
samtools 1.10
Using htslib 1.10.2
===== graphical quality control 2 =====
Java memory size is set to 1200M
Launching application...

QualiMap v.2.2.2-dev
===== quality reports compilation =====
multiqc, version 1.9


---
## <b>I- Some checks as a precaution or Samtools checks</b>

### **1- Available files**

As we change session or even day, let's first check all files are there:

In [4]:
mappedfolder="${gohome}Results/star/"

echo "There are $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam | wc -l) bam files:"
ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam

There are 16 bam files:
/shared/projects/gonseq/Building/Results/star/SRR7430706_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430707_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430708_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430709_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430710_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430711_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430712_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430713_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430738_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430739_Aligned.sortedByCoord.out.bam
/shared/projects/gonseq/Building/Results/star/SRR7430740_Aligned.sortedByCoord.out.bam
/shared/projects/go

### **2- Examining data files: are they what we expect?**

These files are smaller that ``.sam`` files as they are binary ones... but we can't see inside without a specific tool (``samtools view``) included in ``samtools``suite.

In [5]:
samtools --version | head -n 2

samtools 1.10
Using htslib 1.10.2


We list the files in the folder and ask for only the first line (``-n 1``) to get one sample:

In [8]:
abamfile=$(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam | head -n 1)
echo ${abamfile}

/shared/projects/gonseq/Building/Results/star/SRR7430706_Aligned.sortedByCoord.out.bam


In [10]:
samtools view ${abamfile} | head

SRR7430706.14788812	163	chr1	10373	255	50M	=	10391	68	CCTAACCCTAACCCTAACCCCTAACCCCTAACCCTAACCCTAACCCTAAC	C@CFFFFFHGHHHIJJIJIJJJJJJIJJIJIJIIIDDGGIGIIJHJJJJI	NH:i:1	HI:i:1	AS:i:98	nM:i:0	NM:i:0	MD:Z:50	jM:B:c,-1	jI:B:i,-1	MC:Z:50M
SRR7430706.14788812	83	chr1	10391	255	50M	=	10373	-68	CCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC	GHFCA@IIGHHGBIIHAFCIIHIHGIGG@IGIIHGGCGHHFFAFFFF@@@	NH:i:1	HI:i:1	AS:i:98	nM:i:0	NM:i:0	MD:Z:50	jM:B:c,-1	jI:B:i,-1	MC:Z:50M
SRR7430706.21145144	355	chr1	11458	1	1S49M	=	11575	167	GTGGCGGATTATAGGGAAACAACCGGAGCATATGCTGTTTGGTCTCAGTA	CCCFFFFFGHHHHJJJJIIJJJJJJJJJJJJJJJJJJHIJJJEIJJHHHG	NH:i:3	HI:i:1	AS:i:95	nM:i:1	NM:i:1	MD:Z:20C28	jM:B:c,-1	jI:B:i,-1	MC:Z:50M
SRR7430706.21145144	403	chr1	11575	1	50M	=	11458	-167	CCATCAGAATTGTACTGTTCTGTATCCCACCAGCAATGTCTAGGAATGCC	JJJJJJJJIIJJIIJIIGHJJJJJIJIHFJJJJJJJJHHHHHFFFFFCCC	NH:i:3	HI:i:1	AS:i:95	nM:i:1	NM:i:0	MD:Z:50	jM:B:c,-1	jI:B:i,-1	MC:Z:1S49M
SRR7430706.23107087	419	chr1	11633	0	50M	=	11657	74	CACAAAGTGTTTACTTTTGGATTTTT

<ul class="alert alert-block alert-info">
    <li>
        For more information on BAM file format, see <a href="https://biocorecrg.github.io/RNAseq_course_2019/alnpractical.html"><i>BAM/SAM/CRAM format</i></a> section of another training course made by Barcelona's CRG Biocore Facility members on 2019.
    </li>
    <li>
        To understand, or at least know what is inside alignment quality scores, the Broad Institute's tool <a href="https://broadinstitute.github.io/picard/explain-flags.html">Explain SAM Flags</a> may be helpfull!
    </li>
    <li>
        To convert flags between numeric and textual representation, you may also use <code>samtools flags</code>, see <a href="http://www.htslib.org/doc/samtools.html">online manual</a>.
    </li>
</ul>

### **3- Checking `.bam` file's integrity**

You may join us (this pipeline :-D) with your own `.bam` files, either generated from another alignment tool or directly downloaded from network.  
Alternatively, you may have issues (at least I had some several times) with `STAR` and some of your files may be corrupted.  

So, to start (again) off on the right foot, let's check that files are correct. To that purpose, `samtools` will once again help us to deal with `.bam` files.

In [18]:
logfile="${mappedfolder}samtools_quickcheck_bamfiles.log"
echo "Screen output is redirected to ${logfile}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

samtools quickcheck -vvv "${gohome}Results/star/"*_Aligned.sortedByCoord.out.bam \
            &>> ${logfile}
echo "operation ends at $(date)" >> ${logfile}            

# to print user's message (samtools quickcheck is really quick!) and record it
samtools quickcheck "${gohome}Results/star/"*_Aligned.sortedByCoord.out.bam \
            && inbrief="All files are goods." \
            || inbrief="At least one file is corrupted, see logfile."
echo ${inbrief} | tee -a ${logfile}

Screen output is redirected to /shared/projects/gonseq/Building/Results/star/samtools_quickcheck_bamfiles.log
All files are goods.


<div class="alert alert-block alert-warning">
    <b>If there is an issue with your files</b>, you should start again previous steps for corrupted file(s). Indeed, some downstream tools may work but <code>samtools</code> will fail. Besides, you wouldn't be confident with generated results: <b>save time, take some before losing later!</b>
</div>

## <b>II- Some metrics with <code>samtools</code></b>

The commands used for this part belong to a large package of utilities that are very useful to manage those types of files: Samtools (http://www.htslib.org/).

### **1- Preparing command line variables**

An option would be to save results files along with `.bam` files but it's not the best idea to have a lot of files in one folder (especially when saving them, along with long list handling). So we will save the output files to another folder.

In [19]:
samtoolsfolder="${gohome}Results/samtools/"
mkdir -p ${samtoolsfolder}

Count files are *per se* the interesting outputs but, if there is any issue will or for duration information, we nonetheless keep some details into a `.log` file saved with the others reports.

In [20]:
logfolder="${gohome}Results/logfiles/"

### **2- Running command line**

<div class="alert alert-block alert-danger">
    In below loop, we select files which names end with <code>_sortedByCoord.out.bam</code> pattern. <br>
    Indeed, <code>STAR</code> produces a second <code>.bam</code> file if you asked for transcripts counts using <i>TranscriptomeSAM</i> quantification mode. This second filename is suffix with <code>_Aligned.toTranscriptome.out.bam</code>: it's not a standardized one as reads are sorted by transcript name instead of chromosome position. <br>
    <b>If you do not use previous notebook as initially set, please remove or adapt <code>_sortedByCoord.out</code> in below fifth line in order that filename pattern suits your filename format.</b>
</div>

In [23]:
logfile="${logfolder}samtools_stats_flagstat_bamfiles.log"
echo "Screen output is redirected to ${logfile}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

time for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do

    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "========Processing sampleID: ${id}..." | tee -a ${logfile}

    statsfile="${samtoolsfolder}${id}.stats"
    echo "$(date) starting ${statsfile}" >> ${logfile}
    samtools stats ${fn} > ${statsfile}

    flagsfile="${samtoolsfolder}${id}.flagstat"
    echo "$(date) starting ${flagsfile}" >> ${logfile}
    samtools flagstat ${fn} > ${flagsfile}
    
    echo "... done" | tee -a ${logfile}  # caracter string changed after I run it

done

echo "operation ends at $(date)" >> ${logfile}

echo "=== files created after samtools counts ===" >> ${logfile}
ls -lh "${samtoolsfolder}" >> ${logfile}

echo "There are $(ls "${samtoolsfolder}" | wc -l) output files."

Screen output is redirected to /shared/projects/gonseq/Building/Results/logfiles/samtools_stats_flagstat_bamfiles.log
========Processing sampleID: SRR7430706...
... done Thu May 27 11:31:04 CEST 2021 ===
========Processing sampleID: SRR7430707...
... done Thu May 27 11:33:54 CEST 2021 ===
========Processing sampleID: SRR7430708...
... done Thu May 27 11:37:21 CEST 2021 ===
========Processing sampleID: SRR7430709...
... done Thu May 27 11:40:23 CEST 2021 ===
========Processing sampleID: SRR7430710...
... done Thu May 27 11:44:00 CEST 2021 ===
========Processing sampleID: SRR7430711...
... done Thu May 27 11:46:56 CEST 2021 ===
========Processing sampleID: SRR7430712...
... done Thu May 27 11:49:36 CEST 2021 ===
========Processing sampleID: SRR7430713...
... done Thu May 27 11:53:07 CEST 2021 ===
========Processing sampleID: SRR7430738...
... done Thu May 27 11:57:42 CEST 2021 ===
========Processing sampleID: SRR7430739...
... done Thu May 27 12:01:39 CEST 2021 ===
========Processing sam

## <b>III- Some metrics and graphical results with Qualimap</b>

### **1- Tool introduction and preparation step**

We'll run the QUALIMAP program (http://qualimap.conesalab.org/), that collects the data about the `.bam file`, including coverage estimation and many other parameters, and reports a summary of the main properties of the alignment data. QUalimap reads `.sorted.bam` files and generates a folder containing a report on `.html` format.

As you can see from the next command to know which version we are running, Qualimap includes several tools. We will be using the classical bamqc tool which can work on any kind of NGS .bam files. Of note, there is also a rnaqc tool which is dedicated to RNASeq data but it fails to run with C. Parapsilosis annotations.

In [24]:
qualimap --help

Java memory size is set to 1200M
Launching application...

QualiMap v.2.2.2-dev
Built on 2017-08-28 08:37

usage: qualimap <tool> [options]

To launch GUI leave <tool> empty.

Available tools:

    bamqc            Evaluate NGS mapping to a reference genome
    rnaseq           Evaluate RNA-seq alignment data
    counts           Counts data analysis (further RNA-seq data evaluation)
    multi-bamqc      Compare QC reports from multiple NGS mappings
    clustering       Cluster epigenomic signals
    comp-counts      Compute feature counts

Special arguments: 

    --java-mem-size  Use this argument to set Java memory heap size. Example:
                     qualimap bamqc -bam very_large_alignment.bam --java-mem-size=4G



> Remark: QUALIMAP can also be launched on most OS using a command line that will open a user-friendly Java window. On Mac or Windows, you may have to modify the memory RAMs to run it. It is all explained in the documentation.

As previous tools, we will put its results in a `Results/` subfolder:

In [25]:
qualimapfolder="${gohome}Results/qualimap/"
mkdir -p ${qualimapfolder}

An issue with current Qualimap releases is that we can't change file names for both tools we will use.  
Thus, let's create distinct folders for the two tools.

In [26]:
bamqcfolder="${qualimapfolder}bamqc/"
mkdir -p ${bamqcfolder}

In [27]:
rnaseqfolder="${qualimapfolder}rnaseq/"
mkdir -p ${rnaseqfolder}

### **2- General view with <code>qualimap bamqc</code>**

The <code>qualimap</code> options we will use are: <br>
<blockquote>
    <code>-bam filepath</code>, Input mapping file in BAM format sorted by coordinates <br>
    <code>-gff filepath</code> or <code>--feature-file path/to/file</code>, where to find feature file with regions of interest in GFF/GTF or BED format <br>
    <code>-outdir text</code>, Output folder for HTML report and raw data. <br>
    <code>-p VALUE</code> or <code>--sequencing-protocol VALUE</code>, Sequencing library protocol: strand-specific-forward, strand-specific-reverse or non-strand-specific (default). <br>
    <code>-c</code> or <code>--paint-chromosome-limits</code>, Paint chromosome limits inside charts
</blockquote>

Qualimap gives by default an HTML report but we may ask for a PDF or both using <code>-outformat</code> option and corresponding predefined values (PDF, HTML or PDF:HTML for both).

<ul class="alert alert-block alert-info">
    <li>
        See <a href="http://qualimap.conesalab.org/doc_html/analysis.html#bam-qc">online manual</a> for more details
    </li>
</ul>

<div class="alert alert-block alert-danger">
    Following <b>command is prepared for usage on a computational cluster</b> and was developped on the <i>Institut Français de Bioinformatique</i> (IFB)'s core cluster. We use a <i>Large</i> session defined as <b>10 CPU with 50 GB available for RAM</b>. 
</div>

In [ ]:
logfile="${logfolder}qualimap_bamqc_all_samples.log"
echo "Screen output is redirected to ${logfile}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do   
    
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "===== Processing sampleID: ${id}..." | tee -a ${logfile}

    myoutdir="${bamqcfolder}${id}"
    echo "destination folder is ${myoutdir}" >> ${logfile}
    
    echo "qualimap bamqc starts at $(date)" >> ${logfile}
    qualimap bamqc -bam ${fn} \
                    --feature-file "${gohome}Reference/extracted/genome_annotation.gtf" \
                    --sequencing-protocol strand-specific-reverse \
                    --paint-chromosome-limits \
                    -outdir ${myoutdir} \
                    --java-mem-size=20G \
                    &>> ${logfile}
    echo "qualimap bamqc ends at $(date)" >> ${logfile}
    
    echo "... done" | tee -a ${logfile}
  
done 

echo "operation ends at $(date)" >> ${logfile}

echo "=== Files created after qualimap bamqc ===" >> ${logfile}
tree "${bamqcfolder}" >> ${logfile}
echo "Qualimap generated $(find "${bamqcfolder}" -name *.html | wc -l) html reports."

Screen output is redirected to /shared/projects/gonseq/Building/Results/logfiles/qualimap_bamqc_all_samples.log
===== Processing sampleID: SRR7430706...
... done
===== Processing sampleID: SRR7430707...
... done
===== Processing sampleID: SRR7430708...
... done
===== Processing sampleID: SRR7430709...
... done
===== Processing sampleID: SRR7430710...
... done
===== Processing sampleID: SRR7430711...
... done
===== Processing sampleID: SRR7430712...
... done
===== Processing sampleID: SRR7430713...
... done
===== Processing sampleID: SRR7430738...
... done
===== Processing sampleID: SRR7430739...
... done
===== Processing sampleID: SRR7430740...
... done
===== Processing sampleID: SRR7430741...
... done
===== Processing sampleID: SRR7430742...


### **3- Specific RNAseq plots with <code>qualimap rnaseq</code>**

``qualimap rnaseq``, a Qualimap specific option for RNAseq data, gives some more metrics: 
- a *Reads genomic origin* circus-plot to check that reads map to exons
- two *Coverage Profile Along Genes* plots (low and high)
- a *Junction Analysis* circus-plot to distinguish between known and novel junctions

<ul class="alert alert-block alert-info">
    <li>
        See <a href="http://qualimap.conesalab.org/doc_html/analysis.html#rna-seq-qc">online manual for this tool</a> for more details
    </li>
</ul>

#### **3.a- Understanding operations**

With paired end data, we could use following lines:  
<code>qualimap rnaseq -pe \  
               -bam path/to/sortedByNames.bam \ 
               -gtf path/to/gencode_annotation.gtf \ 
               -outdir path/to/outputfolder/</code>

Those options, along with others, are explained in manual:
<blockquote>
     <code>-pe</code> or <code>--paired</code> Setting this flag for paired-end experiments will result in counting fragments instead of reads <br>
     <code>-bam path/to/file.bam</code>, to locate input mapping file in BAM format. <br>
     <code>-gtf path/to/file.gtf</code>, where to find annotations file in Ensembl GTF format (no compressed format). <br>
     <code>-outdir path/to/folder/</code> to define output folder for HTML report and raw data. <br> 
</blockquote>

There are some more options of interest an we will use the first two while letting the third one by default:
<blockquote>
     <code>-p VALUE</code> or <code>--sequencing-protocol VALUE</code>, to precise sequencing library protocol. VALUES among: strand-specific-forward, strand-specific-reverse or non-strand-specific (default) <br>
     <code>-oc path/to/outfile.counts</code>, to place output file for computed counts in a distinct folder. If only name of the file is provided, then the file will be saved in the output folder. <br>
     <code>-a</code> or <code>--algorithm</code>, to set counting algorithm: uniquely-mapped-reads(default) or proportional.
</blockquote>

Contrary to `bamqc`, `rnaseq` Qualimap tool needs alignment files to be sorted by read names. By default, if given otherwise, it will sort `.bam` file... but we already noticed that handling those big files is quite slow with only one process running at a time.  
So we will use `samtools sort` tool to perform it with multiple threads at a time and give `qualimap rnaseq` an already sorted input file and let it know with <code>-s</code> or <code>--sorted</code> (Only required for paired-end analysis...).

In order to follow process, we will use this comman format:  
<code>samtools sort -n \ 
            -@ X -m YG \ 
            -O BAM \ 
            -o path/to/sortedByNames.bam \ 
            -T path/to/temporaryfolder/ \ 
            path/to/input.bam</code>

where options correspond to:  
<blockquote>
    <code>-n</code> for activating sorting by read name (<i>i.e.</i> the QNAME field) <br>
    <code>-o FILE</code>    Write final output to FILE rather than standard output <br>
    <code>-T PREFIX</code>, to write temporary files to a particular folder. Used names are <i>PREFIX.nnnn.bam</i>, where <i>nnnn</i> stands for a 4-digit number <br>
    <code>-O</code> or <code>--output-fmt FORMAT</code> to specify output format (SAM, BAM or CRAM). If not mentionned, <code>samtools</code> will deduce it from file extension (not available if there is no written output file!) <br>
    <code>-@ INT</code> or <code>--threads INT</code>, the number of additional threads to use [0]. Indeed, you will  only use 1 thread by default. <br>
    <code>-m INT</code>, to limit maximum memory per thread; suffix K/M/G recognized [768M]
</blockquote>

#### **3.b- Starting another IFB session**

In [4]:
echo "=== Cell launched on $(date) ==="

echo "=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ==="
jobid=$(squeue -hu $USER | awk '/jupyter/ {print $1}')
sacct --format=JobID,AllocCPUS,NODELIST -j ${jobid}

echo "=== Working's root folder is ==="
gohome="/shared/projects/gonseq/Building/" # to adjust with your project's folder
echo "${gohome}"
echo ""

module load samtools qualimap multiqc

echo "===== quality control & bam sorting ====="
samtools --version | head -n 2
echo "===== graphical quality controls ====="
qualimap --help | head -n 4
echo "===== quality reports compilation ====="
multiqc --version

=== Cell launched on Thu May 27 19:23:19 CEST 2021 ===
=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ===
       JobID  AllocCPUS        NodeList 
------------ ---------- --------------- 
16937696             10     cpu-node-24 
16937696.ba+         10     cpu-node-24 
16937696.0           10     cpu-node-24 
=== Working's root folder is ===
/shared/projects/gonseq/Building/

===== quality control & bam sorting =====
samtools 1.10
Using htslib 1.10.2
===== graphical quality controls =====
Java memory size is set to 1200M
Launching application...

QualiMap v.2.2.2-dev
===== quality reports compilation =====
multiqc, version 1.9


Variables defined in previous session have been forgotten, we need to set them again if we want to use them.

In [5]:
# required: input folder
mappedfolder="${gohome}Results/star/"

# required: output folders
logfolder="${gohome}Results/logfiles/"
qualimapfolder="${gohome}Results/qualimap/"
rnaseqfolder="${qualimapfolder}rnaseq/"

# optional... in case of...
samtoolsfolder="${gohome}Results/samtools/"
bamqcfolder="${qualimapfolder}bamqc/"

#### **3.c- Running loop... with patience**

<div class="alert alert-block alert-warning">
    Below cell will run for <b>at least 30 minutes per sample</b>. Either you come back later or you may use JupyterLab otion to <i>Run selected cell and all below</i> in <i>Run</i> Menu (top bar).
</div>

<div class="alert alert-block alert-danger">
    Following <b>command is prepared for usage on a computational cluster</b> and was developped on the <i>Institut Français de Bioinformatique</i> (IFB)'s core cluster. We use a <i>Large</i> session defined as <b>10 CPU with 50 GB available for RAM</b>. 
</div>

In [ ]:
logfile="${logfolder}qualimap_rnaseq_all-samples.log"
echo "Screen output is redirected to ${logfile}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do  
    
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "===== Processing sampleID: ${id}..." | tee -a ${logfile}
    
    # some preparation
    mytempfile="${rnaseqfolder}${id}_Aligned.sortedByNames.bam"
    myoutdir="${rnaseqfolder}${id}"
    echo "destination folder ${myoutdir}" >> ${logfile}
    
    # bam sorting...
    echo "samtools starts at $(date)" >> ${logfile}
    samtools sort -n \
                --threads 8 -m 5G \
                --output-fmt BAM \
                -o ${mytempfile} \
                -T ${rnaseqfolder} \
                ${fn} \
                &>> ${logfile}
    echo "samtools ends at $(date)" >> ${logfile}

    # some user conversation to help being patient
    echo "...changing tool..." | tee -a ${logfile}

    # then RNAseq QC 
    echo "qualimap rnaseq starts at $(date)" >> ${logfile}
    qualimap rnaseq -bam ${mytempfile} \
                        --sorted \
                        -gtf "/${gohome}Reference/extracted/genome_annotation.gtf" \
                        --sequencing-protocol strand-specific-reverse \
                        --paired \
                        -outdir ${myoutdir} \
                        --java-mem-size=20G \
                        &>> ${logfile}
    echo "qualimap rnaseq ends at $(date)" >> ${logfile}
    
    # removing extra bam file... saving disk space
    rm ${mytempfile}
    
    echo "... done" | tee -a ${logfile}
    
done

echo "operation ends at $(date)" >> ${logfile}

echo "=== Files created after qualimap rnaseq ===" >> ${logfile}
tree "${rnaseqfolder}" >> ${logfile}
echo "Qualimap generated $(find "${rnaseqfolder}" -name *.html | wc -l) html reports." | tee -a ${logfile}

Screen output is redirected to /shared/projects/gonseq/Building/Results/logfiles/qualimap_rnaseq_all-samples.log
===== Processing sampleID: SRR7430706...
... done
===== Processing sampleID: SRR7430707...
...changing tool...
... done
===== Processing sampleID: SRR7430709...
...changing tool...
... done
===== Processing sampleID: SRR7430710...
...changing tool...
... done
===== Processing sampleID: SRR7430711...
...changing tool...
... done
===== Processing sampleID: SRR7430712...
...changing tool...
... done
===== Processing sampleID: SRR7430713...


<div class="alert alert-block alert-danger">
     Equivalent <code>time</code> output for <code>real</code> would be 450min (7h30). <br>
    Cell was launched around 7:30pm (19h30) and continued after I logged off my session (10:25pm, 22h25). Thanks to exported outputs (logfile) I know that cell ended at 03:00am (03h00)... Thus, JupyterLab 12h-session was shut down around 07:00am (07h00) without disturbing process. Nonetheless, I didn't activate below cells (I supposed session would end before <code>qualimap rnaseq</code> ends)... I need to start another - 3rd - session for this notebook.
</div>

## <b>III- Having a summary report with MultiQC</b>

In [4]:
echo "=== Cell launched on $(date) ==="

echo "=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ==="
jobid=$(squeue -hu $USER | awk '/jupyter/ {print $1}')
sacct --format=JobID,AllocCPUS,NODELIST -j ${jobid}

echo "=== Working's root folder is ==="
gohome="/shared/projects/gonseq/Building/" # to adjust with your project's folder
echo "${gohome}"
echo ""

module load samtools qualimap multiqc

echo "===== quality control & bam sorting ====="
samtools --version | head -n 2
echo "===== graphical quality controls ====="
qualimap --help | head -n 4
echo "===== quality reports compilation ====="
multiqc --version

=== Cell launched on Fri May 28 10:29:36 CEST 2021 ===
=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ===
       JobID  AllocCPUS        NodeList 
------------ ---------- --------------- 
16938958             10     cpu-node-36 
16938958.ba+         10     cpu-node-36 
16938958.0           10     cpu-node-36 
=== Working's root folder is ===
/shared/projects/gonseq/Building/

===== quality control & bam sorting =====
samtools 1.10
Using htslib 1.10.2
===== graphical quality controls =====
Java memory size is set to 1200M
Launching application...

QualiMap v.2.2.2-dev
===== quality reports compilation =====
multiqc, version 1.9


In [5]:
# required: input folders
mappedfolder="${gohome}Results/star/"
qualimapfolder="${gohome}Results/qualimap/"
samtoolsfolder="${gohome}Results/samtools/"

# required: output folders
logfolder="${gohome}Results/logfiles/"

When numerous samples are processed, it can easily become tedious to look in each mapping quality report. So we'll run MultiQC (https://multiqc.info/), that scans automatically a folder for all quality checks outputs and produce a single report. MutliQC runs on almost any possible NGS tools (https://multiqc.info/docs/#multiqc-modules).

In [6]:
multiqc --version

multiqc, version 1.9


### **1- Personnalize the report: variables**

Please, specify **file name** you want to have (do not worry about extension, MultiQC will handle this for us) inside quotes in the next cell.  
<b>DO NOT use spaces or any special characters!</b> 

In [7]:
inamemyfile="3_starAlignments-bam-files"

Please, specify a meaningful **title** inside quotes in the next cell, to display at the head of your oncoming report.
<b>No more space limit but still avoid any special characters.</b> 

In [8]:
mytitle="Sorted by coordinates bam files quality for 16 paired end sequenced samples"

Besides, we can add a comment in header's report. It's a good practise to do so. So we will define it in following cell.

> In this cell, we use several lines to keep it readable when displaying notebook. As your text lines are just collapsed together in the html report, be sure to keep last blank space when ending every line).

In [9]:
mycomment=$(echo "Bam files derived from bulk RNA sequencing (human, stranded, rRNA depletion) " \
"performed by Lecluze et al. 2020 (PMID: 32412604, GEO: GSE116278, SRA: SR151462). " \
"Sample set is limited to fetal ovaries and testes at 6 and 12 PCWeeks (n=4 per condition). " \
"SRR7430706-709: female 6 PCW, 709-713: male 6 PCW, 738-741: female 12 PCW, 742-745: male 12 PCW. " \
"Mapping to the reference genome (human, Gencode GRCh38, release 37) performed by Star. Additionnal settings: " \
"Unmapped or too multimapped (>20) are kept in another file, GeneCounts quantification mode activated.")

... and don't forget output folder (I already did this mistake for you ;-) )!

In [11]:
qcsummaries="${gohome}Results/multiqc/"

### **2- Generate summary report**

MultiQC is verbose but, as it will work only on FastQC reports, it is quite short.  *This time, we will have qualimap bamqc, qualimap rnaseq, samtools stats and samtools idxstats*!
So, we will let it lines show in notebook while saving them in a file for later use.

In [12]:
logfile="${logfolder}multiqc-processing_mapped-quality.log"
echo "Screen output is also saved in ${logfile}"

# as time command does not redirect output
echo "operation starting at $(date)" >> ${logfile}
time multiqc --interactive --export \
        --module star ${mappedfolder} \
        --module samtools ${samtoolsfolder} \
        --module qualimap ${qualimapfolder} \
        --outdir "${qcsummaries}" \
        --filename "${inamemyfile}" \
        --title "${mytitle}"  \
        --comment "${mycomment}" \
        "${gohome}Results/" \
        |& tee -a ${logfile}
echo "operation finished at $(date)" >> ${logfile}

# to see which files we have afterward and follow folder sizes
ls -lh "${qcsummaries}" >> ${logfile}
du -h -d1 "${gohome}Results/" >> ${logfile}

Screen output is also saved in /shared/projects/gonseq/Building/Results/logfiles/multiqc-processing_mapped-quality.log
[WARNING]         multiqc : MultiQC Version v1.10.1 now available!
[INFO   ]         multiqc : This is MultiQC v1.9
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Report title: Sorted by coordinates bam files quality for 16 paired end sequenced samples
[INFO   ]         multiqc : Searching   : /shared/projects/gonseq/Building/Results/star
[INFO   ]         multiqc : Searching   : /shared/projects/gonseq/Building/Results/samtools
[INFO   ]         multiqc : Searching   : /shared/projects/gonseq/Building/Results/qualimap
[INFO   ]         multiqc : Searching   : /shared/projects/gonseq/Building/Results
[INFO   ]         multiqc : Only using modules star, samtools, qualimap
[INFO   ]        qualimap : Found 16 BamQC reports
/shared/ifbstor1/software/miniconda/envs/multiqc-1.9/lib/python3.6/site-packages/multiqc/plots/bargraph.py:451: UserWar

### **3- Get an eye on used disk space**

In [14]:
du -h -d2 ${gohome}

23M	/shared/projects/gonseq/Building/Results/multiqc
8.4M	/shared/projects/gonseq/Building/Results/fastp
3.8M	/shared/projects/gonseq/Building/Results/samtools
488K	/shared/projects/gonseq/Building/Results/logfiles
34M	/shared/projects/gonseq/Building/Results/fastqc
117G	/shared/projects/gonseq/Building/Results/star
4.0K	/shared/projects/gonseq/Building/Results/.ipynb_checkpoints
38M	/shared/projects/gonseq/Building/Results/qualimap
118G	/shared/projects/gonseq/Building/Results
154G	/shared/projects/gonseq/Building/Data/fastq
46G	/shared/projects/gonseq/Building/Data/sra
44K	/shared/projects/gonseq/Building/Data/info
36K	/shared/projects/gonseq/Building/Data/.ipynb_checkpoints
199G	/shared/projects/gonseq/Building/Data
4.0K	/shared/projects/gonseq/Building/.ipynb_checkpoints
220K	/shared/projects/gonseq/Building/Pipeline/.ipynb_checkpoints
440K	/shared/projects/gonseq/Building/Pipeline
1.4G	/shared/projects/gonseq/Building/Reference/extracted
28G	/shared/projects/gonseq/Building/Refere

---
___

Now we go on to count genes and/or transcripts expression levels.

**=> Step 6: Features counting** 

___